In [1]:
import numpy as np
import sklearn as sk
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import seaborn as sns
import re
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib as plt

## Reading in the data

In [2]:
df_raw = pd.read_csv("data/boardgames1.csv")
df_raw.head()

,objectid,name,yearpublished,sortindex,minplayers,maxplayers,minplaytime,maxplaytime,minage,min_community,...,boardgameartist,boardgamepublisher,boardgamehonor,boardgamecategory,boardgameversion,boardgamemechanic,boardgameexpansion,boardgamefamily,description,gamelink
0,174430,Gloomhaven,2017,1,1,4,60,120,12,3.0,...,"['Alexandr Elichev', 'Josh T. McDowell', 'Alva...","['Cephalofair Games', 'Albi', 'Asmodee', 'Feue...",['2017 Best Science Fiction or Fantasy Board G...,"['Adventure', 'Exploration', 'Fantasy', 'Fight...","['Chinese edition', 'Czech edition', 'English ...","['Campaign / Battle Card Driven', 'Cooperative...","['Gloomhaven: Forgotten Circles', 'Gloomhaven:...","['Campaign Games', 'Components: Miniatures', '...",Gloomhaven is a game of Euro-inspired tactica...,/boardgame/174430/gloomhaven
1,161936,Pandemic Legacy Season 1,2015,2,2,4,60,60,13,4.0,...,['Chris Quilliams'],"['Z-Man Games', 'Asterion Press', 'Devir', 'Fi...",['2015 Cardboard Republic Immersionist Laurel ...,"['Environmental', 'Medical']","['Chinese blue edition', 'Chinese red edition'...","['Action Points', 'Cooperative Game', 'Hand Ma...",['None'],"['Campaign Games', 'Legacy', 'Pandemic']",Pandemic Legacy is a co-operative campaign gam...,/boardgame/161936/pandemic-legacy-season-1
2,167791,Terraforming Mars,2016,3,1,5,120,120,12,3.0,...,['Isaac Fryxelius'],"['FryxGames', 'Arclight', 'Fantasmagoria', 'Gh...",['2016 Cardboard Republic Architect Laurel Nom...,"['Economic', 'Environmental', 'Industry / Manu...","['Bulgarian edition', 'Chinese edition', 'Czec...","['Card Drafting', 'End Game Bonuses', 'Hand Ma...",['French Championship Promo Cards (fan expansi...,"['Fryxgames Future Timeline', 'Planets: Mars',...","In the 2400s, mankind begins to terraform the ...",/boardgame/167791/terraforming-mars
3,182028,Through the Ages A New Story of Civilization,2015,4,2,4,120,120,14,3.0,...,"['Filip Murmak', 'Radim Pech', 'Jakub Politzer...","['Czech Games Edition', 'Cranio Creations', 'D...",['2015 Golden Geek Best Strategy Board Game No...,"['Card Game', 'Civilization', 'Economic']","['Chinese edition', 'Czech edition', 'English ...","['Action Points', 'Auction/Bidding', 'Auction:...",['Through the Ages: New Leaders and Wonders'],"['Tableau Building', 'Through the Ages']",Through the Ages: A New Story of Civilization ...,/boardgame/182028/through-ages-new-story-civil...
4,224517,Brass Birmingham,2018,5,2,4,60,120,14,3.0,...,"['Lina Cossette', 'David Forest', 'Damien Mamm...","['Roxley', 'BoardM Factory', 'Conclave Editora...",['2018 Golden Geek Best Board Game Artwork & P...,"['Economic', 'Industry / Manufacturing', 'Tran...","['English deluxe edition', 'English retail edi...","['Hand Management', 'Income', 'Loans', 'Market...",['None'],"['Beer', 'Brass', 'Cities: Birmingham (England...",Brass: Birmingham is an economic strategy game...,/boardgame/224517/brass-birmingham


## Preprocessing

First we drop a lot of columns that don't contain relevant information, or information that is not applicable for our business case. Since we are developing games, information that can only be obtained after the relase is not relevant. (For example awards,comments, podcasts).

In [3]:
drop_cols = ["objectid","name","yearpublished","sortindex","totalvotes","languagedependence","baverage","numgeeklists","numtrading","numwanting",
             "numcomments","siteviews","numplays","numplays_month",
             "news","blogs","weblink","podcast","boardgamehonor_cnt","boardgameexpansion_cnt", "boardgameversion_cnt","boardgamefamily_cnt","boardgamedesigner","boardgameartist",
             "boardgamepublisher","boardgamehonor","boardgameversion",
            "boardgameexpansion","boardgamefamily","description","gamelink","playerage","min_community","max_community","totalvotes","label"]
df = df_raw.drop(drop_cols,axis=1)
df.columns

Index(['minplayers', 'maxplayers', 'minplaytime', 'maxplaytime', 'minage',
       'usersrated', 'average', 'stddev', 'avgweight', 'numweights',
       'boardgamedesigner_cnt', 'boardgameartist_cnt',
       'boardgamepublisher_cnt', 'boardgamecategory_cnt',
       'boardgamemechanic_cnt', 'boardgamecategory', 'boardgamemechanic'],
      dtype='object')

Additionaly, information on the game mechanics and category might be important therefore we will extract and N-hot encode them. 

In [4]:
def extract_and_encode(df1,column):
    df = df1.copy(deep=True)
    df[column] = df[column].apply(lambda x: re.sub(",,",",",re.sub("[\[\]' ]","",x)).split(","))

    mlb = MultiLabelBinarizer(sparse_output=False)


    df = df.join(pd.DataFrame(mlb.fit_transform(df.pop(column)),
                index=df.index,columns=mlb.classes_),
                lsuffix='_mechanic', rsuffix='_category')
    return df

df = extract_and_encode(df,column="boardgamemechanic")
df = extract_and_encode(df,column="boardgamecategory")


In order not to increase dimensionality too much, each mechanic or category must be present in at least 500 games. (Or in 2.5% of games)

In [5]:
freq = df.iloc[:,16:].sum()

condition = [True]*16
condition = np.concatenate((np.array(condition),(freq > 500).to_numpy()))

df = df.loc[:, condition]

## Outlier removal

When looking at the maximum we can see that our data includes a few outliers in different variables

In [6]:
df.iloc[:,0:15].describe(include="all")

,minplayers,maxplayers,minplaytime,maxplaytime,minage,usersrated,average,stddev,avgweight,numweights,boardgamedesigner_cnt,boardgameartist_cnt,boardgamepublisher_cnt,boardgamecategory_cnt,boardgamemechanic_cnt
count,20000.000000,20000.00000,20000.000000,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,2.055250,5.59215,68.096450,94.28945,9.476350,739.616350,6.275440,1.499691,1.931761,48.059050,1.339200,1.379250,2.500100,2.568050,2.306900
std,0.745537,15.04921,466.502106,1005.75196,3.738842,3096.843206,1.065339,0.340743,0.897206,198.113638,0.787003,4.768926,5.082961,1.366515,1.707764
min,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,2.000000,4.00000,20.000000,30.00000,8.000000,47.000000,5.712778,1.309920,1.255050,4.000000,1.000000,0.000000,1.000000,2.000000,1.000000
50%,2.000000,4.00000,30.000000,45.00000,10.000000,105.000000,6.354170,1.471575,1.910900,9.000000,1.000000,1.000000,1.000000,2.000000,2.000000
75%,2.000000,6.00000,60.000000,90.00000,12.000000,333.250000,6.955793,1.663782,2.500000,25.000000,2.000000,2.000000,2.000000,3.000000,3.000000
max,10.000000,999.00000,60000.000000,120000.00000,25.000000,90730.000000,9.442860,4.500000,5.000000,7104.000000,21.000000,508.000000,174.000000,14.000000,18.000000


In [7]:
for column in df.iloc[:,0:15]:
    counter=1
    percentage = 1
    while percentage > 0.02:
        counter += 1
        percentage = (df.loc[:,column] >= counter).sum()/20000 
    print("{}% of {} is over {}".format(round(100*percentage,1),column,counter))

0.8% of minplayers is over 5
2.0% of maxplayers is over 15
0.5% of minplaytime is over 361
1.2% of maxplaytime is over 361
1.3% of minage is over 17
2.0% of usersrated is over 7336
0.1% of average is over 9
0.3% of stddev is over 3
1.8% of avgweight is over 4
2.0% of numweights is over 456
1.8% of boardgamedesigner_cnt is over 4
1.6% of boardgameartist_cnt is over 7
1.8% of boardgamepublisher_cnt is over 13
1.2% of boardgamecategory_cnt is over 7
1.2% of boardgamemechanic_cnt is over 8


Now we remove the (at most) top 2% of different variables.

In [8]:
tmp = pd.DataFrame({'a': df["maxplayers"] > 15,'b': df["minplaytime"] > 361,'c': df["maxplaytime"] > 361,'d': df["boardgamedesigner_cnt"] > 4,'e': df["boardgameartist_cnt"] > 7,'f': df["boardgamepublisher_cnt"] > 13})
print("By removing outliers we lose {}% of data".format(100*np.any(tmp,axis=1).sum()/20000))
df_trimmed = df.loc[~np.any(tmp,axis=1),:]

By removing outliers we lose 6.09% of data


## Exporting the data




In [9]:
#df_trimmed.to_csv("data/preprocessed.csv")